**[2019.11.21.] Azure와 함께하는 AutoML || SBA 빅데이터 사이언스 실무연계 프로젝트 || 전미정**

# Azure와 함께하는 Automated Machine Learning_GPU 💻
**Naver 영화 리뷰 감정 분석 모델**

<img src="http://dl.dropbox.com/s/mkya7ana2z0y2vt/Screen%20Shot%202019-10-02%20at%2012.50.29%20PM.png" width="800">



* **Part1. 개발 환경 설정**  
* **Part2. 학습 데이터 가져오기**  
* **Part3. 모델 학습**  
* **Part4. 모델 결과**  
* **Part5. 모델 테스트**  

---

## Part1. 개발 환경 설정

1. Workspace 연결
2. Experiment 생성
3. 학습에 사용할 remote compute 생성
4. RunConfig 설정

In [ ]:
!pip install azureml-sdk[notebooks,automl]==1.0.57

### 1-1. Workspace 연결

In [ ]:
import logging

from matplotlib import pyplot as plt
import pandas as pd
import os

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig

# Azure core SDK version 확인
print("Azure ML SDK Version: ", azureml.core.VERSION)

In [ ]:
# config.json에서 복붙해주세요!

subscription_id = 'subscription_id'
resource_group  = 'resource_group'
workspace_name  = 'workspace_name'

try:
    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
    ws.write_config()
    print('Library configuration succeeded')
except:
    print('Workspace not found')
    

### 1-2. Experiment 생성

In [ ]:
# 실험명 및 프로젝트 파일 디렉토리 설정
experiment_name = 'automl-classification-movie-review'

experiment=Experiment(ws, experiment_name)

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

### 1-3.학습에 사용할 remote compute 생성

* Azure Machine Learning(Aml) compute 생성에 5분 가량 소요됩니다.  
* GPU를 활용하기 위해서는 리소스 region을 EastUS로 설정해주세요!


**참고자료**  
[Azure Machine Learning에서 계산 대상이란?](https://docs.microsoft.com/ko-kr/azure/machine-learning/service/concept-compute-target/?WT.mc_id=AI-MVP-5003262)  
[Azure Machine Learning compute 설정하기](https://docs.microsoft.com/ko-kr/azure/machine-learning/service/how-to-set-up-training-targets/?WT.mc_id=AI-MVP-5003262)

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

# 생성할 Compute cluster 이름 지정
amlcompute_cluster_name = "gpucluster"

found = False
# 이미 동일한 이름의 compute cluster 존재 여부 확인
cts = ws.compute_targets
if amlcompute_cluster_name in cts and cts[amlcompute_cluster_name].type == 'AmlCompute':
    found = True
    print('Found existing compute target.')
    compute_target = cts[amlcompute_cluster_name]
    
if not found:
    print('Creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_NC12",
                                                                # STANDARD_NC12 ==  NVIDIA Tesla K80 GPU
                                                                max_nodes = 4)

    # compute 생성
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, provisioning_config)
    
    compute_target.wait_for_completion(show_output = True, min_node_count = None, timeout_in_minutes = 20)

print(compute_target.get_status())

### 1-4. RunConfig 생성 및 설정


In [ ]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
import pkg_resources

# python 기반의 RunConfig 객체 생성
conda_run_config = RunConfiguration(framework="python")

# 1-3에서 생성한 compute target 설정
conda_run_config.target = compute_target
conda_run_config.environment.docker.enabled = True

cd = CondaDependencies.create(conda_packages=['numpy','py-xgboost<=0.80'])
conda_run_config.environment.python.conda_dependencies = cd
print('Conda setting finished')

---

## Part2. 학습 데이터 가져오기

네이버 영화 리뷰 데이터 가져오기

* [Lucy Park님의 github](https://github.com/e9t/nsmc) 데이터를 활용했습니다.
* 전체 데이터 200K 중 30K를 학습 데이터로 사용했습니다.
* 데이터를 불러오는데 약 5분 가량 소요됩니다.

In [ ]:
# Dataset 형태로 데이터 가져오기 🌟⭐️🌟
data = 'https://www.dropbox.com/s/n6kfxnsymw28ov6/review_train_30K.csv?dl=1'
dataset = Dataset.Tabular.from_delimited_files(data)
dataset.take(10).to_pandas_dataframe()

In [ ]:
# X_train/y_train으로 데이터 분류
X_train = dataset.drop_columns(columns=['label'])
y_train = dataset.keep_columns(columns=['label'], validate=True)

---

## Part3. 모델 학습

`AutoMLConfig` 객체 생성

Experiment를 실행하는데 사용되는 설정값 및 데이터 정의

|Property|Description|
|-|-|
|**task**|classification/regression/forecast|
|**X**|(sparse) array-like, shape = [n_samples, n_features]|
|**y**|(sparse) array-like, shape = [n_samples, ], 다중 클래스|
|**primary_metric**|<i>accuracy</i><br><i>AUC_weighted</i><br><i>average_precision_score_weighted</i><br><i>norm_macro_recall</i><br><i>precision_score_weighted</i>|
|**iteration_timeout_minutes**|iteration 1회 소요 시간제한(분)|
|**max_time_sec**|최대 학습 시간(초)(will be deprecated)|
|**iterations**|학습 반복 횟수|
|**preprocess**|데이터 전처리 여부|
|**blacklist_models**|사용하지 않을 모델|
|**whitelist_models**|꼭 사용할 모델|


**primary_metric에 대한 자세한 내용은 [여기](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-auto-train#primary-metric/?WT.mc_id=AI-MVP-5003262)를 참고해주세요** 

**Task에 따른 model 종류는 [여기](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-auto-train#select-your-experiment-type/?WT.mc_id=AI-MVP-5003262)를 참고해주세요**

In [ ]:
whitelist_models=["SGD"]

automl_settings = {
    "iterations": 5,
    "n_cross_validations": 2,
    "primary_metric": 'AUC_weighted',
    "preprocess": True,
    "max_concurrent_iterations": 5,
    "verbosity": logging.INFO,
    "whitelist_models" : whitelist_models
}

automl_config = AutoMLConfig(task = 'classification',
                             debug_log = 'automl_errors.log',
                             run_configuration=conda_run_config,
                             X = X_train,
                             y = y_train,
                             **automl_settings
                            )

In [ ]:
# submit method
remote_run = experiment.submit(automl_config, show_output = True)

In [ ]:
# remote computre의 run 객체를 취소하고 싶을 때 아래 코드를 사용하세요.

# remote_run.cancel()
# print(remote_run.status)

---

## Part4. 학습 결과

`widgets`은 1분에 한번씩 업데이트됩니다.  
위젯 아래쪽에 세부 정보를 확인할 수 있는 링크가 제공됩니다.

In [ ]:
# Azure Run 위젯 실행
from azureml.widgets import RunDetails
RunDetails(remote_run).show() 

In [ ]:
# 베스트 모델 확인
best_run, fitted_model = remote_run.get_output()
print("Best Model:", best_run)
print("=========")
print("Fitted Model:", fitted_model)

---

## Part5. 모델 테스트

In [ ]:
# 테스트 데이터 불러오기
data = "https://www.dropbox.com/s/akq6a9h7mmmv8px/review_test_0.5K.csv?dl=1"
dataset = Dataset.Tabular.from_delimited_files(data)
dataset.take(10).to_pandas_dataframe()

In [ ]:
X_test = dataset.drop_columns(columns=['label'])
y_test = dataset.keep_columns(columns=['label'], validate=True)

In [ ]:
X_test_df = X_test.to_pandas_dataframe()
y_test_df = y_test.to_pandas_dataframe()

In [ ]:
import numpy as np

y_pred = fitted_model.predict(X_test_df)
actual = np.array(y_test_df)
actual = actual[:,0]
X_test_df = X_test_df['document']
print(y_pred.shape, " ", actual.shape)

In [ ]:
# 랜덤으로 10개 선택 및 테스트
count = 0
sample_size = 10
for index in np.random.choice(len(actual), sample_size, replace = False):
    count = count + 1
    label = actual[index]
    review = X_test_df[index]
    pred = y_pred[index]
    print("리뷰: {} \n 예측 값: {} || 실제 값: {}".format(review, pred, label))

리뷰 분석 모델! 정말 재미있지 않나요? 🤗 👏🏻 🥳

---